In [ ]:
#install relevant packages

install.packages("dplyr")
library(dplyr)
library(ggplot2)
install.packages("ivreg")
library(ivreg)
install.packages("modelsummary")
library(modelsummary)
install.packages("tableone")
library(tableone)
install.packages("kableExtra")
library(kableExtra)
install.packages("sensemakr")
library(sensemakr)

#display

options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)


In [52]:
#load original dataset
dil=read.csv("dilex.csv")

In [53]:
#filter, remove outliers, and recode some variables

dil=dil%>%
filter(AGE>=18)%>% #only include patients >=18 years of age
#I will assume that if value for post-op opioids is missing, then if a pain score is present for that time, then the missing value indicates that 
#zero opioids were given.
mutate(MSO4_EQUIV_DOSE_IV_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_IV_DAY1))%>% 
mutate(MSO4_EQUIV_DOSE_IV_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_IV_DAY2))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_PO_DAY1))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_PO_DAY2))%>%
mutate(EQUIV_MSO4_IV_DOSE_60=ifelse(is.na(EQUIV_MSO4_IV_DOSE_60)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_60))%>%
mutate(EQUIV_MSO4_IV_DOSE_120=ifelse(is.na(EQUIV_MSO4_IV_DOSE_120)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_120))%>%
mutate(POSTOP_MSO4_EQUIV_IV_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_IV_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_IV_TOTAL))%>%
mutate(POSTOP_MSO4_EQUIV_PO_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_PO_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_PO_TOTAL))%>%
mutate(FENTANYL_MG=ifelse(is.na(FENTANYL_MG),0,FENTANYL_MG))%>%
#when fentanyl is missing, it means that 0 fentanyl was given intraoperatively
mutate(ANES_TYPE_HANDOFF=ifelse(ANES_TYPE_HANDOFF=="GENERAL",1,0))%>%
#I create a variable "general" which is 1 if the patient had only general, and 0 otherwise
filter(HYDROMORPHONE_MG<40)%>%
#There were a few errors in diaudid doses, I removed values where dilaudid was >40 and value where it was exactly 11.2 
filter(HYDROMORPHONE_MG!=11.2)%>%
filter(BMI<120)%>% #120 #as these are likely errors where weight went into the BMI field
filter(UCLA_LOC_GROUP=="RR OR"|UCLA_LOC_GROUP=="SM OR")%>%
filter(DURATION<1500)%>%


mutate(case=1)%>%
mutate(case=ifelse(CASE_SRV_NAME=="Dentistry-Hospital"|CASE_SRV_NAME=="Pediatrics, Hematology and Oncology"|
                   CASE_SRV_NAME=="Medicine, Hematology and Oncology"|CASE_SRV_NAME=="Ophthalmology"|
                   CASE_SRV_NAME=="Podiatry"|CASE_SRV_NAME=="Radiology"|CASE_SRV_NAME=="Medicine, Gastroenterology"
                   |CASE_SRV_NAME=="Medicine, Pulmonary Disease",2,case))%>%
mutate(case=ifelse(CASE_SRV_NAME=="Surgery, Cardiac",0,case))

In [ ]:
#how much missing data in the new dataset
dil%>%
summarise_all(funs(sum(is.na(.))))

In [54]:
dil0=dil
#create saved version of dataset before removing dates

dil=dil%>%
filter(DAY_SINCE_INTERVENTION<142)%>%
#filter(DAY_SINCE_INTERVENTION>-142)%>%
filter(DAY_SINCE_INTERVENTION>-250)%>%
mutate(cat=0)%>%
mutate(cat=ifelse(DAY_SINCE_INTERVENTION>=0 &DAY_SINCE_INTERVENTION<=142,1,cat))




In [ ]:
#save temp before create table 1
dilt=dil

#This cell will create a table 1


## Make categorical variables factors
varsToFactor <- c("case","SEX","ASA_SCORE","ASA_EMERGENT","ANES_TYPE_HANDOFF","UCLA_LOC_GROUP",
                  "KETAMINE_GTT")
dil[varsToFactor] <- lapply(dil[varsToFactor], factor)

dil=dil%>%
rename("Weight(kg)" = WEIGHT_KG, "Duration(min)"=DURATION, "ASA Score"=ASA_SCORE, "ASA(E)"=ASA_EMERGENT, Ketamine=KETAMINE_GTT, "Fentanyl(mg)"=
FENTANYL_MG, "Morphine(mg)"=MORPHINE_MG, "Anesthesia type"=ANES_TYPE_HANDOFF, "Sex(female)"=SEX, "Surgery Type"=case)

## Create a variable list

vars <- c("Weight(kg)","BMI", "Duration(min)", "AGE", "Fentanyl(mg)","Morphine(mg)",
         "Anesthesia type","Sex(female)", "ASA Score", "ASA(E)",
                  "Ketamine", "Surgery Type")



## Create Table 1 stratified by trt
tableOne <- CreateTableOne(vars = vars, strata = c("cat"), data = dil)
p <- print(tableOne,  printToggle = FALSE, noSpaces = TRUE)
#p <- print(tableOne, nonnormal = c('BMI', 'Fentanyl(mg)','Morphine(mg)', 'Duration(min)'), printToggle = FALSE, noSpaces = TRUE)
kable(p, format = "latex", booktabs=TRUE)

#turn back to dil
dil=dilt




In [36]:
#transformation so that effect is for 0.2mg 
dil=dil%>%
mutate(HYDROMORPHONE_MG=HYDROMORPHONE_MG*5)

In [37]:
#create variable daytot1 and daytot2 to be combined IV and PO opioids 
dil=dil%>%
mutate(daytot1=MSO4_EQUIV_DOSE_IV_DAY1+MSO4_EQUIV_DOSE_PO_DAY1)%>%
mutate(daytot2=MSO4_EQUIV_DOSE_IV_DAY2+MSO4_EQUIV_DOSE_PO_DAY2)

In [ ]:
#simple regressions

firstpain=lm(FIRST_PAIN_SCORE~HYDROMORPHONE_MG, data=dil)


lastpain=lm(LAST_PAIN_SCORE~HYDROMORPHONE_MG, data=dil)


morp60=lm(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG, data=dil)


morp120=lm(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG, data=dil)




maxpain1=lm(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG, data=dil)

maxpain2=lm(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG, data=dil)

avepain1=lm(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG, data=dil)

avpain2=lm(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG, data=dil)

mortot1=lm(daytot1~HYDROMORPHONE_MG, data=dil)

mortot2=lm(daytot2~HYDROMORPHONE_MG, data=dil)

models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("HYDROMORPHONE_MG"="")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


In [ ]:
#multivariable regression
firstpain=lm(FIRST_PAIN_SCORE~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)


lastpain=lm(LAST_PAIN_SCORE~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)


morp60=lm(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)


morp120=lm(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)


morptot=lm(POSTOP_MSO4_EQUIV_IV_TOTAL~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

maxpain1=lm(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

maxpain2=lm(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

avepain1=lm(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

avpain2=lm(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

mortot1=lm(daytot1~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)

mortot2=lm(daytot2~HYDROMORPHONE_MG+(ASA_SCORE)+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case, data=dil)




models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("case"="Surgery Type", "HYDROMORPHONE_MG"="Hydromorphone (mg)", "ASA_SCORE"="ASA Score", "ASA_EMERGENT"= "ASA(E)",
"AGE"="Age", "FENTANYL_MG"="Fentanyl (mg)", "DURATION"="Duration", "WEIGHT_KG"= "Weight_kg")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")

In [ ]:
#TSLS analysis without covariates

firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG |cat, data=dil)

morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG |cat, data=dil)

morptot=ivreg(POSTOP_MSO4_EQUIV_IV_TOTAL~HYDROMORPHONE_MG |cat, data=dil)

maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG |cat, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG |cat, data=dil)




mortot1=ivreg(daytot1~HYDROMORPHONE_MG |cat, data=dil)
mortot2=ivreg(daytot2~HYDROMORPHONE_MG |cat, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("HYDROMORPHONE_MG"="")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")



In [ ]:
#output for tsls with covariates
firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG+case|cat +ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case ,data=dil)


lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)


morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)


morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)




maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

mortot1=ivreg(daytot1~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)

mortot2=ivreg(daytot2~HYDROMORPHONE_MG+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case|cat+ASA_SCORE+ASA_EMERGENT+WEIGHT_KG+BMI+as.factor(ANES_TYPE_HANDOFF)+DURATION+AGE+as.factor(SEX)+
FENTANYL_MG +case, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    #"MME PACU total"=morptot,

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)



coef_rename=c("case"="Surgery Type", "HYDROMORPHONE_MG"="Hydromorphone (mg)", "ASA_SCORE"="ASA Score", "ASA_EMERGENT"= "ASA(E)",
"AGE"="Age", "FENTANYL_MG"="Fentanyl (mg)", "DURATION"="Duration", "WEIGHT_KG"= "Weight_kg")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")




In [ ]:
#output for fentanyl covariate only

firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG +FENTANYL_MG|cat +FENTANYL_MG ,data=dil)


lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)


morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)


morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)




maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

mortot1=ivreg(daytot1~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)

mortot2=ivreg(daytot2~HYDROMORPHONE_MG+FENTANYL_MG|cat+FENTANYL_MG, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    #"MME PACU total"=morptot,

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)


coef_rename=c("HYDROMORPHONE_MG"="Hydromorphone(mg)", "FENTANYL_MG"="Fentanyl(mg)")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")




**Sensitivity Analysis**


Time windows

In [ ]:
#for analysis with time window extending back 500 days


#load original dataset
dil=read.csv("dilex.csv")
#filter, remove outliers, and recode some variables

dil=dil%>%
filter(AGE>=18)%>% #only include patients >=18 years of age
#I will assume that if value for post-op opioids is missing, then if a pain score is present for that time, then the missing value indicates that 
#zero opioids were given.
mutate(MSO4_EQUIV_DOSE_IV_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_IV_DAY1))%>% 
mutate(MSO4_EQUIV_DOSE_IV_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_IV_DAY2))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_PO_DAY1))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_PO_DAY2))%>%
mutate(EQUIV_MSO4_IV_DOSE_60=ifelse(is.na(EQUIV_MSO4_IV_DOSE_60)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_60))%>%
mutate(EQUIV_MSO4_IV_DOSE_120=ifelse(is.na(EQUIV_MSO4_IV_DOSE_120)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_120))%>%
mutate(POSTOP_MSO4_EQUIV_IV_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_IV_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_IV_TOTAL))%>%
mutate(POSTOP_MSO4_EQUIV_PO_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_PO_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_PO_TOTAL))%>%
mutate(FENTANYL_MG=ifelse(is.na(FENTANYL_MG),0,FENTANYL_MG))%>%
#when fentanyl is missing, it means that 0 fentanyl was given intraoperatively
mutate(ANES_TYPE_HANDOFF=ifelse(ANES_TYPE_HANDOFF=="GENERAL",1,0))%>%
#I create a variable "general" which is 1 if the patient had only general, and 0 otherwise
filter(HYDROMORPHONE_MG<40)%>%
#There were a few errors in diaudid doses, I removed values where dilaudid was >40 and value where it was exactly 11.2 
filter(HYDROMORPHONE_MG!=11.2)%>%
filter(BMI<120)%>% #120 #as these are likely errors where weight went into the BMI field
filter(UCLA_LOC_GROUP=="RR OR"|UCLA_LOC_GROUP=="SM OR")%>%
filter(DURATION<1500)%>%


mutate(case=1)%>%
mutate(case=ifelse(CASE_SRV_NAME=="Dentistry-Hospital"|CASE_SRV_NAME=="Pediatrics, Hematology and Oncology"|
                   CASE_SRV_NAME=="Medicine, Hematology and Oncology"|CASE_SRV_NAME=="Ophthalmology"|
                   CASE_SRV_NAME=="Podiatry"|CASE_SRV_NAME=="Radiology"|CASE_SRV_NAME=="Medicine, Gastroenterology"
                   |CASE_SRV_NAME=="Medicine, Pulmonary Disease",2,case))%>%
mutate(case=ifelse(CASE_SRV_NAME=="Surgery, Cardiac",0,case))


dil=dil%>%
filter(DAY_SINCE_INTERVENTION<142)%>%
#filter(DAY_SINCE_INTERVENTION>-142)%>%
filter(DAY_SINCE_INTERVENTION>-500)%>%
mutate(cat=0)%>%
mutate(cat=ifelse(DAY_SINCE_INTERVENTION>=0 &DAY_SINCE_INTERVENTION<=142,1,cat))

#transformation so that effect is for 0.2mg 
dil=dil%>%
mutate(HYDROMORPHONE_MG=HYDROMORPHONE_MG*5)

#create variable daytot1 and daytot2 to be combined IV and PO opioids 
dil=dil%>%
mutate(daytot1=MSO4_EQUIV_DOSE_IV_DAY1+MSO4_EQUIV_DOSE_PO_DAY1)%>%
mutate(daytot2=MSO4_EQUIV_DOSE_IV_DAY2+MSO4_EQUIV_DOSE_PO_DAY2)

#TSLS analysis without covariates

firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG |cat, data=dil)

morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG |cat, data=dil)

morptot=ivreg(POSTOP_MSO4_EQUIV_IV_TOTAL~HYDROMORPHONE_MG |cat, data=dil)

maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG |cat, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG |cat, data=dil)




mortot1=ivreg(daytot1~HYDROMORPHONE_MG |cat, data=dil)
mortot2=ivreg(daytot2~HYDROMORPHONE_MG |cat, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("HYDROMORPHONE_MG"="")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")





In [ ]:
#for analysis with time window extending back 142 days

#load original dataset
dil=read.csv("dilex.csv")
#filter, remove outliers, and recode some variables

dil=dil%>%
filter(AGE>=18)%>% #only include patients >=18 years of age
#I will assume that if value for post-op opioids is missing, then if a pain score is present for that time, then the missing value indicates that 
#zero opioids were given.
mutate(MSO4_EQUIV_DOSE_IV_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_IV_DAY1))%>% 
mutate(MSO4_EQUIV_DOSE_IV_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_IV_DAY2))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_PO_DAY1))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_PO_DAY2))%>%
mutate(EQUIV_MSO4_IV_DOSE_60=ifelse(is.na(EQUIV_MSO4_IV_DOSE_60)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_60))%>%
mutate(EQUIV_MSO4_IV_DOSE_120=ifelse(is.na(EQUIV_MSO4_IV_DOSE_120)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_120))%>%
mutate(POSTOP_MSO4_EQUIV_IV_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_IV_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_IV_TOTAL))%>%
mutate(POSTOP_MSO4_EQUIV_PO_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_PO_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_PO_TOTAL))%>%
mutate(FENTANYL_MG=ifelse(is.na(FENTANYL_MG),0,FENTANYL_MG))%>%
#when fentanyl is missing, it means that 0 fentanyl was given intraoperatively
mutate(ANES_TYPE_HANDOFF=ifelse(ANES_TYPE_HANDOFF=="GENERAL",1,0))%>%
#I create a variable "general" which is 1 if the patient had only general, and 0 otherwise
filter(HYDROMORPHONE_MG<40)%>%
#There were a few errors in diaudid doses, I removed values where dilaudid was >40 and value where it was exactly 11.2 
filter(HYDROMORPHONE_MG!=11.2)%>%
filter(BMI<120)%>% #120 #as these are likely errors where weight went into the BMI field
filter(UCLA_LOC_GROUP=="RR OR"|UCLA_LOC_GROUP=="SM OR")%>%
filter(DURATION<1500)%>%


mutate(case=1)%>%
mutate(case=ifelse(CASE_SRV_NAME=="Dentistry-Hospital"|CASE_SRV_NAME=="Pediatrics, Hematology and Oncology"|
                   CASE_SRV_NAME=="Medicine, Hematology and Oncology"|CASE_SRV_NAME=="Ophthalmology"|
                   CASE_SRV_NAME=="Podiatry"|CASE_SRV_NAME=="Radiology"|CASE_SRV_NAME=="Medicine, Gastroenterology"
                   |CASE_SRV_NAME=="Medicine, Pulmonary Disease",2,case))%>%
mutate(case=ifelse(CASE_SRV_NAME=="Surgery, Cardiac",0,case))

dil=dil%>%
filter(DAY_SINCE_INTERVENTION<142)%>%
#filter(DAY_SINCE_INTERVENTION>-142)%>%
filter(DAY_SINCE_INTERVENTION>-142)%>%
mutate(cat=0)%>%
mutate(cat=ifelse(DAY_SINCE_INTERVENTION>=0 &DAY_SINCE_INTERVENTION<=142,1,cat))

#transformation so that effect is for 0.2mg 
dil=dil%>%
mutate(HYDROMORPHONE_MG=HYDROMORPHONE_MG*5)

#create variable daytot1 and daytot2 to be combined IV and PO opioids 
dil=dil%>%
mutate(daytot1=MSO4_EQUIV_DOSE_IV_DAY1+MSO4_EQUIV_DOSE_PO_DAY1)%>%
mutate(daytot2=MSO4_EQUIV_DOSE_IV_DAY2+MSO4_EQUIV_DOSE_PO_DAY2)

#TSLS analysis without covariates

firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG |cat, data=dil)

morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG |cat, data=dil)

morptot=ivreg(POSTOP_MSO4_EQUIV_IV_TOTAL~HYDROMORPHONE_MG |cat, data=dil)

maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG |cat, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG |cat, data=dil)




mortot1=ivreg(daytot1~HYDROMORPHONE_MG |cat, data=dil)
mortot2=ivreg(daytot2~HYDROMORPHONE_MG |cat, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("HYDROMORPHONE_MG"="")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")





In [ ]:
#for analysis with time window extended to include cohort 3

#load original dataset
dil=read.csv("dilex.csv")
#filter, remove outliers, and recode some variables

dil=dil%>%
filter(AGE>=18)%>% #only include patients >=18 years of age
#I will assume that if value for post-op opioids is missing, then if a pain score is present for that time, then the missing value indicates that 
#zero opioids were given.
mutate(MSO4_EQUIV_DOSE_IV_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_IV_DAY1))%>% 
mutate(MSO4_EQUIV_DOSE_IV_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_IV_DAY2))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_PO_DAY1))%>%
mutate(MSO4_EQUIV_DOSE_PO_DAY2=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY2)&!is.na(MAX_PAIN_SCORE_DAY2),0,MSO4_EQUIV_DOSE_PO_DAY2))%>%
mutate(EQUIV_MSO4_IV_DOSE_60=ifelse(is.na(EQUIV_MSO4_IV_DOSE_60)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_60))%>%
mutate(EQUIV_MSO4_IV_DOSE_120=ifelse(is.na(EQUIV_MSO4_IV_DOSE_120)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_120))%>%
mutate(POSTOP_MSO4_EQUIV_IV_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_IV_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_IV_TOTAL))%>%
mutate(POSTOP_MSO4_EQUIV_PO_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_PO_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_PO_TOTAL))%>%
mutate(FENTANYL_MG=ifelse(is.na(FENTANYL_MG),0,FENTANYL_MG))%>%
#when fentanyl is missing, it means that 0 fentanyl was given intraoperatively
mutate(ANES_TYPE_HANDOFF=ifelse(ANES_TYPE_HANDOFF=="GENERAL",1,0))%>%
#I create a variable "general" which is 1 if the patient had only general, and 0 otherwise
filter(HYDROMORPHONE_MG<40)%>%
#There were a few errors in diaudid doses, I removed values where dilaudid was >40 and value where it was exactly 11.2 
filter(HYDROMORPHONE_MG!=11.2)%>%
filter(BMI<120)%>% #120 #as these are likely errors where weight went into the BMI field
filter(UCLA_LOC_GROUP=="RR OR"|UCLA_LOC_GROUP=="SM OR")%>%
filter(DURATION<1500)%>%


mutate(case=1)%>%
mutate(case=ifelse(CASE_SRV_NAME=="Dentistry-Hospital"|CASE_SRV_NAME=="Pediatrics, Hematology and Oncology"|
                   CASE_SRV_NAME=="Medicine, Hematology and Oncology"|CASE_SRV_NAME=="Ophthalmology"|
                   CASE_SRV_NAME=="Podiatry"|CASE_SRV_NAME=="Radiology"|CASE_SRV_NAME=="Medicine, Gastroenterology"
                   |CASE_SRV_NAME=="Medicine, Pulmonary Disease",2,case))%>%
mutate(case=ifelse(CASE_SRV_NAME=="Surgery, Cardiac",0,case))

dil=dil%>%
filter(DAY_SINCE_INTERVENTION<250)%>%
#filter(DAY_SINCE_INTERVENTION>-142)%>%
filter(DAY_SINCE_INTERVENTION>-250)%>%
mutate(cat=0)%>%
mutate(cat=ifelse(DAY_SINCE_INTERVENTION>=0 &DAY_SINCE_INTERVENTION<=142,1,cat))


#transformation so that effect is for 0.2mg 
dil=dil%>%
mutate(HYDROMORPHONE_MG=HYDROMORPHONE_MG*5)

#create variable daytot1 and daytot2 to be combined IV and PO opioids 
dil=dil%>%
mutate(daytot1=MSO4_EQUIV_DOSE_IV_DAY1+MSO4_EQUIV_DOSE_PO_DAY1)%>%
mutate(daytot2=MSO4_EQUIV_DOSE_IV_DAY2+MSO4_EQUIV_DOSE_PO_DAY2)

#TSLS analysis without covariates

firstpain=ivreg(FIRST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

lastpain=ivreg(LAST_PAIN_SCORE~HYDROMORPHONE_MG |cat, data=dil)

morp60=ivreg(EQUIV_MSO4_IV_DOSE_60~HYDROMORPHONE_MG |cat, data=dil)

morp120=ivreg(EQUIV_MSO4_IV_DOSE_120~HYDROMORPHONE_MG |cat, data=dil)

morptot=ivreg(POSTOP_MSO4_EQUIV_IV_TOTAL~HYDROMORPHONE_MG |cat, data=dil)

maxpain1=ivreg(MAX_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

maxpain2=ivreg(MAX_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

avepain1=ivreg(AVG_PAIN_SCORE_DAY1~HYDROMORPHONE_MG |cat, data=dil)

avpain2=ivreg(AVG_PAIN_SCORE_DAY2~HYDROMORPHONE_MG |cat, data=dil)

morp1=ivreg(MSO4_EQUIV_DOSE_IV_DAY1~HYDROMORPHONE_MG |cat, data=dil)

morp2=ivreg(MSO4_EQUIV_DOSE_IV_DAY2~HYDROMORPHONE_MG |cat, data=dil)




mortot1=ivreg(daytot1~HYDROMORPHONE_MG |cat, data=dil)
mortot2=ivreg(daytot2~HYDROMORPHONE_MG |cat, data=dil)


models <- list(
  "1st PACU Pain"     = firstpain,
  "Last PACU Pain" = lastpain,    
    "Ave Pain (day 1)"=avepain1,
   "Ave Pain (day 2)"=avpain2,
     "Max Pain (day 1)"=maxpain1,
    "Max Pain (day 2)"=maxpain2
    

)
models1<- list(

"MME PACU 60min" =morp60,
    "MME PACU 120min"=morp120,
    

     'MME day1'=mortot1,
     'MME day2'=mortot2
   
)

coef_rename=c("HYDROMORPHONE_MG"="")
modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")


modelsummary(models1,estimate   =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

  coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, coef_rename=coef_rename, output = "latex")





Contour Plots

In [ ]:
#simply change the outcome variable to perform all the analyses

model=lm(FIRST_PAIN_SCORE~BMI+cat, data=dil)
summary(model)


In [ ]:
sens=sensemakr(model=model, treatment="cat", benchmark_covariates = "BMI" ,kd=c(1,5,10,20,40))
summary(sens)

In [ ]:
plot(sens, label.text=FALSE,sensitivity.of = "t-value", cex.label.text = 0.0001)

*Falfification Tests*

In [64]:


#load the dataset with not hydromorphone given intraop
dil1=read.csv("dilno.csv")

#perform same transformations, except for day 2 which we do not have
dil1=dil1%>%
mutate(MSO4_EQUIV_DOSE_IV_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_IV_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_IV_DAY1))%>% 

mutate(MSO4_EQUIV_DOSE_PO_DAY1=ifelse(is.na(MSO4_EQUIV_DOSE_PO_DAY1)&!is.na(MAX_PAIN_SCORE_DAY1),0,MSO4_EQUIV_DOSE_PO_DAY1))%>%

mutate(EQUIV_MSO4_IV_DOSE_60=ifelse(is.na(EQUIV_MSO4_IV_DOSE_60)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_60))%>%
mutate(EQUIV_MSO4_IV_DOSE_120=ifelse(is.na(EQUIV_MSO4_IV_DOSE_120)&!is.na(FIRST_PAIN_SCORE),0,EQUIV_MSO4_IV_DOSE_120))%>%
mutate(POSTOP_MSO4_EQUIV_IV_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_IV_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_IV_TOTAL))%>%
mutate(POSTOP_MSO4_EQUIV_PO_TOTAL=ifelse(is.na(POSTOP_MSO4_EQUIV_PO_TOTAL)&!is.na(FIRST_PAIN_SCORE),0,POSTOP_MSO4_EQUIV_PO_TOTAL))%>%
mutate(FENTANYL_MG=ifelse(is.na(FENTANYL_MG),0,FENTANYL_MG))%>%
#when fentanyl is missing, it means that 0 fentanyl was given intraoperatively
mutate(ANES_TYPE_HANDOFF=ifelse(ANES_TYPE_HANDOFF=="GENERAL",1,0))%>%
#I create a variable "general" which is 1 if the patient had only general, and 0 otherwise
filter(BMI<120)%>% #120 #as these are likely errors where weight went into the BMI field
filter(UCLA_LOC_GROUP=="RR OR"|UCLA_LOC_GROUP=="SM OR")%>%
filter(DURATION<1500)%>%


mutate(case=1)%>%
mutate(case=ifelse(CASE_SRV_NAME=="Dentistry-Hospital"|CASE_SRV_NAME=="Pediatrics, Hematology and Oncology"|
                   CASE_SRV_NAME=="Medicine, Hematology and Oncology"|CASE_SRV_NAME=="Ophthalmology"|
                   CASE_SRV_NAME=="Podiatry"|CASE_SRV_NAME=="Radiology"|CASE_SRV_NAME=="Medicine, Gastroenterology"
                   |CASE_SRV_NAME=="Medicine, Pulmonary Disease",2,case))%>%
mutate(case=ifelse(CASE_SRV_NAME=="Surgery, Cardiac",0,case))


In [65]:
dil1=dil1%>%
filter(DAY_SINCE_INTERVENTION<142)%>%
filter(DAY_SINCE_INTERVENTION>-250)%>%
mutate(cat=0)%>%
mutate(cat=ifelse(DAY_SINCE_INTERVENTION>=0 &DAY_SINCE_INTERVENTION<=142,1,cat))


In [66]:


firstpain=(lm(FIRST_PAIN_SCORE~cat,data=dil1))
lastpain=(lm(LAST_PAIN_SCORE~cat,data=dil1))
avepain1=(lm(AVG_PAIN_SCORE_DAY1~cat,data=dil1))

maxpain1=(lm(MAX_PAIN_SCORE_DAY1~cat,data=dil1))




firstpaina=(lm(FIRST_PAIN_SCORE~cat,data=dil))
lastpaina=(lm(LAST_PAIN_SCORE~cat,data=dil))
avepain1a=(lm(AVG_PAIN_SCORE_DAY1~cat,dat=dil))
maxpain1a=(lm(MAX_PAIN_SCORE_DAY1~cat,data=dil))

In [67]:
models <- list(
  "firstpain"     = firstpain,
  "Last Pain" = lastpain,    
    "avepain1"=avepain1,

    'maxpain1'=maxpain1
)

models1 <- list(
  "1st Pain"     = firstpaina,
  "Last Pain" = lastpaina,    
    "avepain1"=avepain1a,

    'maxpain1'=maxpain1a
  
)


In [ ]:

modelsummary(models,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, output = "latex")


modelsummary(models1,estimate  =   "Estimate= {estimate}",
  statistic = c("CI={conf.low}, {conf.high}","p = {p.value}"),

coef_omit = "Intercept",gof_omit = 'R2|R2 Adj|F|AIC|BIC|Log.Lik', fmt=2, output = "latex")